In [1]:
pip install transformers torch gradio

In [3]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [5]:
# Load pre-trained chatbot model from Hugging Face
# You can use a smaller model like "microsoft/DialoGPT-small" for demo
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# Function to generate chatbot response
def ice_cream_chatbot(user_input, chat_history=[]):
    # Encode the user input and append chat history
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append history if exists
    bot_input_ids = torch.cat([torch.LongTensor(chat_history), new_input_ids], dim=-1) if chat_history else new_input_ids

    # Generate response
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decode response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Update chat history
    chat_history.append(new_input_ids)

    return response, chat_history_ids

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🍦 Ice Cream Chatbot")

    with gr.Row():
        with gr.Column():
            chatbot_output = gr.Chatbot(label="Ice Cream Chatbot")
            user_input = gr.Textbox(placeholder="Ask me about ice cream flavors...", label="Your Message")
            submit_btn = gr.Button("Send")

    # Function to handle sending message
    def respond(message, chat_history):
        response, chat_history_ids = ice_cream_chatbot(message, chat_history)
        return [(message, response)], chat_history_ids

    submit_btn.click(respond, inputs=[user_input, chatbot_output], outputs=[chatbot_output, chatbot_output])

/tmp/ipython-input-129644275.py:26: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_output = gr.Chatbot(label="Ice Cream Chatbot")
/tmp/ipython-input-129644275.py:26: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot_output = gr.Chatbot(label="Ice Cream Chatbot")


In [11]:
# Launch app (Hugging Face Spaces will auto-run this)
if __name__ == "__main__":
    demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b27890d45959f9e6d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 